In [4]:
%load_ext autoreload
%autoreload 2
import tqdm, tqdm.notebook
tqdm.tqdm = tqdm.notebook.tqdm  # notebook-friendly progress bars
from pathlib import Path
import time
import os
from hloc import extract_features, match_features, reconstruction, visualization, pairs_from_exhaustive
from hloc.visualization import plot_images, read_image
from hloc.utils import viz_3d
ways = ['spsg', 'r2d2nn', 'spnn', 'siftnn']
way = ways[3]
images = Path('./Data/imgwithmask')
outputs = Path('./Data/output/'+way)
!rm -rf $outputs
sfm_pairs = outputs / 'pairs-sfm.txt'
loc_pairs = outputs / 'pairs-loc.txt'
sfm_dir = outputs / 'sfm'
features = outputs / 'features.h5'
matches = outputs / 'matches.h5'
if way == 'spsg':
    feature_conf = extract_features.confs['superpoint_aachen']
    matcher_conf = match_features.confs['superglue']
elif way == 'r2d2nn':
    feature_conf = extract_features.confs['r2d2']
    matcher_conf = match_features.confs['NN-mutual']
elif way == 'spnn':
    feature_conf = extract_features.confs['superpoint_aachen']
    matcher_conf = match_features.confs['NN-mutual']
elif way == 'siftnn':
    feature_conf = extract_features.confs['sift']
    matcher_conf = match_features.confs['NN-mutual']

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/bin/zsh: /home/a410/anaconda3/envs/hloc/lib/libtinfo.so.6: no version information available (required by /bin/zsh)


In [5]:
references = [p.relative_to(images).as_posix() for p in (images / '').iterdir()]
print(len(references), "mapping images")
#plot_images([read_image(images / r) for r in references[:4]], dpi=50)

33 mapping images


In [6]:
extract_features.main(feature_conf, images, image_list=references, feature_path=features)
pairs_from_exhaustive.main(sfm_pairs, image_list=references)
match_features.main(matcher_conf, sfm_pairs, features=features, matches=matches)

[2023/05/11 19:46:29 hloc INFO] Extracting local features with configuration:
{'model': {'name': 'dog'},
 'output': 'feats-sift',
 'preprocessing': {'grayscale': True, 'resize_max': 1600}}


  0%|          | 0/33 [00:00<?, ?it/s]

[2023/05/11 19:46:32 hloc INFO] Finished exporting features.
[2023/05/11 19:46:32 hloc INFO] Found 528 pairs.
[2023/05/11 19:46:32 hloc INFO] Matching local features with configuration:
{'model': {'do_mutual_check': True, 'name': 'nearest_neighbor'},
 'output': 'matches-NN-mutual'}


  0%|          | 0/528 [00:00<?, ?it/s]

[2023/05/11 19:47:10 hloc INFO] Finished exporting matches.


PosixPath('Data/output/siftnn/matches.h5')